# Setting up for BCG identification

If you're a student working on a BCG identification project, there is no need for you to run this notebook!

You're welcome to read through and get a handle on what the setup processes are doing though - the point of this notebook is essentially to make sure that all optical, infrared, and X-ray images are generated/downloaded ready for you to look at. The 'spot the BCG' notebook (the first of the set) will explain everything you need to know about those different wavelengths, and what we use them for.

**If you're running this to setup a BCG identification run, make sure to go and configure everything in 'ident_run_setup.py' before running!**

## Import Statements

In [1]:
from ident_run_setup import cosmo, rel_miss, rel_downloaders, side_length, init_samp_file, HISTORY_FILE_PATH, \
    HISTORY_ROOT, load_history, proj_name, update_history, OUTPUT_ROOT, OUTPUT_CLUSTER_PATH

import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from tqdm import tqdm
from astropy.units import Quantity
import os
from warnings import warn
import json
from copy import deepcopy
from shutil import copyfile
from urllib.error import HTTPError

try:
    from xga.sourcetools import ang_to_rad
    from xga.imagetools.misc import pix_deg_scale
except:
    # Using a general 'except' clause is not usually recommended - you ideally want to be catching
    #  specific TYPES of error, so would specify them after the 'except' clause
    raise ValueError("There was a problem importing XGA - likely running with a pre-multi-mission version.")

# Store the current directory
TOP_WD = os.getcwd()

## What cosmology is set?

In [2]:
cosmo

LambdaCDM(name=None, H0=<Quantity 70. km / (Mpc s)>, Om0=0.3, Ode0=0.7, Tcmb0=<Quantity 0. K>, Neff=3.04, m_nu=None, Ob0=None)

## Loading and setting up the sample of clusters

### Reading in the sample file and checking for required information

In [3]:
# Point the notebook to the CSV containing basic information about the sample of clusters
samp = pd.read_csv(init_samp_file)

# A notebook isn't really the ideal environment for this, as each individual cell can be re-run, changing what is stored in 
#  memory (RAM). As such, we'll run the checks for specific column names in the same cell in which we read in the sample file, so 
#  no other cell has a chance to modify the dataframe
if (~np.isin(np.array(['name', 'cent_im_ra', 'cent_im_dec', 'redshift']), samp.columns)).any():
    col_names = ", ".join(samp.columns)
    raise KeyError("Some required sample columns are not present; input file columns are - {}.".format(col_names))

# Show a snippet of the sample as a quick check, though it must have the required columns to have reached this point
samp.head(5)

,name,cent_im_ra,cent_im_dec,redshift,r500,r500-,r500+,r2500,r2500-,r2500+,XCS_NAME
0,SDSSXCS-124,0.800578,-6.091818,0.247483,1181.028159,21.202221,23.202641,534.834740,7.579124,7.591855,XMMXCS J000312.1-060530.5
1,SDSSXCS-2789,0.955540,2.068019,0.105285,1007.860978,17.194150,17.201505,438.706515,5.198301,5.213676,XMMXCS J000349.3+020404.8
2,SDSSXCS-290,2.722639,29.161021,0.348495,913.052256,30.878754,31.209675,412.606577,11.014644,11.199164,XMMXCS J001053.4+290939.6
3,SDSSXCS-1018,4.406325,-0.876192,0.214403,902.259231,22.444665,23.366414,399.213342,6.774208,6.817562,XMMXCS J001737.5-005234.2
4,SDSSXCS-134,4.908390,3.609818,0.277304,1123.320736,19.219312,19.225964,510.738163,8.475349,7.419581,XMMXCS J001938.0+033635.3


### Adding angular-to-proper distance ratios to sample

In [4]:
ang_step = Quantity(1, 'arcmin')
ang_prop_ratios = (ang_to_rad(ang_step, samp['redshift'].values, cosmo=cosmo) / ang_step)
# Unfortunately dataframes and quantities do not seem to be getting on at the moment, so we save the
#  float version of these values
samp['ang_prop_ratio'] = ang_prop_ratios.value
ang_prop_ratios

<Quantity [232.85996665, 115.80299749, 295.55703817, 208.96598069,
           252.89890756, 269.81894828, 247.33755558, 114.52471534,
           194.94231833, 170.37909225, 267.73456559, 243.03871063,
           237.04613445, 250.97065817, 296.03651206, 218.71296151,
           226.99764355, 139.9268915 , 162.36287491, 291.81585769,
           293.38496704, 193.45924487, 263.67045471, 266.85508032,
           279.93736808, 154.01150752, 159.03597111, 172.18417051,
           196.33890555, 150.35262411, 215.1828343 , 292.29413403,
           270.50009972, 189.55288372, 188.8036126 , 224.0695333 ,
           182.51841177, 185.28558211, 162.42377759, 278.45876006,
           223.99993875, 265.06180711, 196.64751522, 198.76749964,
           111.08449251, 236.60234346, 237.22957694, 174.26959646,
           218.20911517, 224.5003265 , 270.38729628, 263.68781889,
           267.22414102, 186.1694538 , 226.32939904, 274.73269229,
           281.46620208, 115.51995152, 201.97544287, 260.42878

## Setting up directory structure and history files

### Storage for files that record the history of the identification run

In [5]:
# Switch the root path to a relative path, otherwise the history is not portable across machines
static_samp_file = os.path.relpath(os.path.join(HISTORY_ROOT, 'STATIC_'+proj_name+'.csv'))

if os.path.exists(HISTORY_FILE_PATH):
    cur_history = load_history()
    samp = pd.read_csv(static_samp_file)
    ang_prop_ratios = Quantity(samp['ang_prop_ratio'], 'kpc/arcmin')
    
else:
    if not os.path.exists(HISTORY_ROOT):
        os.makedirs(HISTORY_ROOT)
    
    if not os.path.exists(HISTORY_FILE_PATH):
        data_ops = {n: {'raw_images': {mn: {'complete': False, 'unavailable': False} for mn in rel_miss}} for n in samp['name'].values}
        bcg_ops = {n: {'ident_complete': False} for n in samp['name'].values}
        
        cur_history = {'project_name': proj_name,
                       'num_clusters': len(samp),
                       'static_samp_file': static_samp_file,
                       'chosen_missions': rel_miss,
                       'cosmo_repr': str(cosmo),
                       'side_length': side_length.to('kpc').value,
                       'data_operations': data_ops,
                       'bcg_identification': bcg_ops,
                       'bcg_spec_identification': bcg_ops}
        
        with open(HISTORY_FILE_PATH, 'w') as write_historo:
            json.dump(cur_history, write_historo)
        
        samp.to_csv(static_samp_file, index=False)


### Storage for downloaded/generated raw images

In [6]:
raw_im_pth_abs = os.path.abspath("raw_images/{n}/{m}") + '/'
raw_im_pth = os.path.relpath(raw_im_pth_abs)

for src_name in samp['name'].values:
    for miss_name in rel_miss:
        if not os.path.exists(raw_im_pth.format(m=miss_name, n=src_name)):
            os.makedirs(raw_im_pth.format(m=miss_name, n=src_name))

### Storage for outputs

In [7]:
for src_name in samp['name'].values:
    if not os.path.exists(OUTPUT_CLUSTER_PATH.format(n=src_name)):
        os.makedirs(OUTPUT_CLUSTER_PATH.format(n=src_name))

## Generating and saving XMM images

We address XMM observations slightly differently, as one of the best ways of getting images is to download the raw data and reprocess it - as this is complex, we also allow for the introduction of already-generated FITS images to the history.

Currently, existing file names must be formatted in a very specific way (see the code for further information).

In [8]:
# existing_xmm = None
existing_xmm = "../../useful_data/riley_bcg_ident_xmm_comb_final/"

# The sub-directory path and file name we expect for existing XMM data
im_file_existing_xmm = "{n}/xmm/{n}_0.5-2.0keV_comb.fits"

In [9]:
if 'xmm' in rel_miss:
    if existing_xmm is not None:

        # Check that the existing XMM path is real
        if not os.path.exists(existing_xmm):
            raise FileNotFoundError("The specified 'existing_xmm' path does not exist.")

        # We make a copy of the data operations section of the history, as it may be changed in this next bit
        data_op_hist = deepcopy(cur_history['data_operations'])
        # And a change flag
        hist_change = False
        
        with tqdm(total=len(samp), desc='Moving existing XMM images') as onwards:
            for row_ind, row in samp.iterrows():
                # Read out the cluster name - so we can store things in the right place, and keep track of which image
                #  belongs to which cluster
                cur_name = row['name']
                
                # First, lets check if this cluster has had XMM data set up already - if yes then we can skip
                if cur_history['data_operations'][cur_name]['raw_images']['xmm']['complete']:
                    onwards.update(1)
                    continue
    
                # We currently don't build any flexibility into the directory and file name format supported for 
                #  adding existing images - that will likely change if this gets developed further
                cur_exist_path = os.path.join(existing_xmm, im_file_existing_xmm.format(n=cur_name))
                if not os.path.exists(cur_exist_path):
                    data_op_hist[cur_name]['raw_images']['xmm']['unavailable'] = True
                    data_op_hist[cur_name]['raw_images']['xmm']['complete'] = True
                    hist_change = True
                    
                else:
                    cur_final_dir = raw_im_pth.format(n=cur_name, m='xmm')
                    final_xmm_im_path = os.path.join(cur_final_dir, "{n}_0.5-2.0keV_comb.fits".format(n=cur_name))
    
                    copyfile(cur_exist_path, final_xmm_im_path)
                                                     
                    # data_op_hist[cur_name]['raw_images']['xmm']['arcsec_per_pix'] = 
                    # data_op_hist[cur_name]['raw_images']['xmm']['cen_pos'] = 
                    data_op_hist[cur_name]['raw_images']['xmm']['complete'] = True
                    data_op_hist[cur_name]['raw_images']['xmm']['unavailable'] = False
                    # Make sure we save the RELATIVE path - otherwise this history file won't work on other systems
                    data_op_hist[cur_name]['raw_images']['xmm']['im_path'] = os.path.relpath(final_xmm_im_path)
                    hist_change = True
                    
                onwards.update(1)

        # Only run the update history function if there were changes
        if hist_change:
            cur_history = update_history({'data_operations': data_op_hist})
        
    else:
        raise NotImplementedError("The downloading and processing of XMM data for BCG identification "\
                                  "projects has not yet been implemented.")

Moving existing XMM images: 100%|██████████████████████████████████████████| 150/150 [00:00<00:00, 362.87it/s]


## Downloading DESI Legacy Survey optical/NIR photometry

In [10]:
#
ls_pixsize_lim = Quantity(3000, 'pix')

#
ls_start_pix_scale = Quantity(0.262, 'arcsec/pixel')

In [11]:
if 'desi-ls' in rel_miss:
    # Setting up the downloader class instance
    desi_down = rel_downloaders['desi-ls']()

    ls_pix_side_lengths = ((side_length/ang_prop_ratios)/ls_start_pix_scale).to('pix')

    if (ls_pix_side_lengths > ls_pixsize_lim).any():
        ls_worst_pix_len = ls_pix_side_lengths.max()
        ls_pix_scale_multi = np.ceil(ls_worst_pix_len / ls_pixsize_lim).astype(int)
        warn("The Legacy Survey pixel side length for one or more clusters is greater than allowed by the cutout " \
             "server - changing pixel scale to {}".format(ls_pix_scale_multi*ls_start_pix_scale), stacklevel=2)
    else:
        ls_pix_scale_multi = 1

    # Set up the final pixel scale
    ls_final_pix_scale = ls_start_pix_scale * ls_pix_scale_multi

    # Recalculate with the final scale
    ls_pix_side_lengths = ((side_length/ang_prop_ratios)/ls_final_pix_scale).to('pix')

    # We make a copy of the data operations section of the history, as it may be changed in this next bit
    data_op_hist = deepcopy(cur_history['data_operations'])
    # And a change flag
    hist_change = False
    
    # Iterating through dataframes like this is a bad idea if you're performing calculations on dataframe information, or if the
    #  dataframe is large (more than a few thousand entries), as iterating is very slow. However, we're just iterating to read
    #  things out, so it is sort of okay (and quite convenient)
    with tqdm(total=len(samp), desc='Downloading DESI Legacy Survey DR10 photometry') as onwards:
        for row_ind, row in samp.iterrows():
            # Read out the cluster name - so we can store things in the right place, and keep track of which image
            #  belongs to which cluster
            cur_name = row['name']
            
            # First, lets check if this cluster has had DESI-LS data downloaded already. We'll use the project 
            #  history, as it is much computationally cheaper than checking if the expected file names exist
            if cur_history['data_operations'][cur_name]['raw_images']['desi-ls']['complete']:
                onwards.update(1)
                continue

            # Reading out relevant information - position is obviously very important, it is where we will center the
            #  downloaded images
            cur_ra = row['cent_im_ra']
            cur_dec = row['cent_im_dec']
            # The conversion between arcmin and arcminutes we calculated earlier, note that retrieving from a row
            #  of a dataframe like this seems to render it a float again, rather than an Astropy quantity
            cur_ang_prop_ratio = Quantity(row['ang_prop_ratio'], 'kpc/arcmin')
    
            # Though we already calculated the final pixel sizes we need for the LS images, the download method
            #  wants the pixel scale and an angular side length specified seperately, so we have to convert the
            #  proper side-length to degrees
            cur_deg_side_length = (side_length / cur_ang_prop_ratio).to('deg')
    
            # Populate the raw image storage directory path, with the cluster name and the current mission name
            cur_download_dir = raw_im_pth.format(n=cur_name, m='desi-ls')
            # Set up the final raw image file name, with some useful info in it
            cur_file_name = "{n}_sidelength{sl}_pixscale{ps}.jpeg".format(n=cur_name, sl=str(side_length).replace(' ', ''), 
                                                                          ps=str(ls_final_pix_scale).replace(' ', '').replace('/', 'per'))
            # Combine the raw image storage path with the file name
            cur_download_pth = os.path.join(cur_download_dir, cur_file_name)
    
            desi_down.download(ra=cur_ra, dec=cur_dec, bands='griz', layer='ls-dr10', mode='jpeg', autoscale=False, 
                               ddir=cur_download_dir, size=cur_deg_side_length.value, pixscale=ls_final_pix_scale.value)
    
            # Change the name of the downloaded file to something more descriptive
            down_file_name = 'legacystamps_{r}_{d}_ls-dr10.jpeg'.format(r='{:.6f}'.format(cur_ra), d='{:.6f}'.format(cur_dec))
            down_file_path = os.path.join(cur_download_dir, down_file_name)
            os.rename(down_file_path, cur_download_pth)

            # Now we add to the data operation history - as well as setting the change flag to True so we know to run the 
            #  history file update at the end of the looping
            hist_change = True
            data_op_hist[cur_name]['raw_images']['desi-ls']['complete'] = True
            data_op_hist[cur_name]['raw_images']['desi-ls']['arcsec_per_pix'] = ls_final_pix_scale.value
            # This shouldn't be necessary, as it should be setup so the sample content won't change, but we'll
            #  save the central RA and Dec here as well, so all the WCS-related information will be in one place
            data_op_hist[cur_name]['raw_images']['desi-ls']['cen_pos'] = [cur_ra, cur_dec]
            # Make sure we save the RELATIVE path - otherwise this history file won't work on other systems
            data_op_hist[cur_name]['raw_images']['desi-ls']['im_path'] = os.path.relpath(cur_download_pth)
            
            onwards.update(1)

    # Only run the update history function if there were changes
    if hist_change:
        cur_history = update_history({'data_operations': data_op_hist})

# TODO COULD ADD SOME NICE FAIL SAFE SO THAT IT ATTEMPTS TO DOWNLOAD AN SDSS IMAGE IF THERE IS NO LS DATA

/Users/dt237/software/anaconda3/envs/new_xga_dev/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: The Legacy Survey pixel side length for one or more clusters is greater than allowed by the cutout server - changing pixel scale to 0.524 arcsec / pix
  exec(code_obj, self.user_global_ns, self.user_ns)



  9%|██████▎                                                                 | 32/368 [00:00<00:05, 62.73KB/s]
369KB [00:00, 556.57KB/s]                                                                                     



 54%|████████████████████████████████████▊                               | 844/1561 [00:00<00:00, 1268.84KB/s]
1562KB [00:00, 1743.39KB/s]                                                                                   



 12%|████████▌                                                              | 32/267 [00:00<00:00, 298.77KB/s]
268KB [00:00, 1015.79KB/s]                                                                                    



 36%|█████████████████████████▌                                            | 176/483 [00:00<00:00, 913.88KB/s]
484KB [00:00, 1049.73KB/s]                                                                                    

...


 13%|█████████▌                                                             | 32/239 [00:00<00:00, 315.13KB/s]
240KB [00:00, 709.66KB/s]                                                                                     



 60%|█████████████████████████████████████████▊                            | 148/248 [00:00<00:00, 440.15KB/s]
249KB [00:00, 469.69KB/s]                                                                                     



 52%|████████████████████████████████████▋                                 | 148/282 [00:00<00:00, 481.16KB/s]
283KB [00:00, 559.87KB/s]                                                                                     



 72%|██████████████████████████████████████████████████                    | 302/422 [00:01<00:00, 253.89KB/s]
423KB [00:01, 282.88KB/s]                                                                                     



 73%|███████████████████████████████████████████████████▏                  | 256/350 [00:00<00:00, 395.05KB/s]
351KB [00:01, 327.56KB/s]                                                                                     

## Downloading VLASS images

In [12]:
if 'vlass' in rel_miss:
    # Setting up the downloader class instance for VLASS - it has fewer download options than the DESI one does
    vlass_down = rel_downloaders['vlass'](datatype='se')

    # We make a copy of the data operations section of the history, as it may be changed in this next bit
    data_op_hist = deepcopy(cur_history['data_operations'])
    # And a change flag
    hist_change = False

    os.chdir('raw_images/')
    
    # Iterating through dataframes like this is a bad idea if you're performing calculations on dataframe information, or if the
    #  dataframe is large (more than a few thousand entries), as iterating is very slow. However, we're just iterating to read
    #  things out, so it is sort of okay (and quite convenient)
    with tqdm(total=len(samp), desc='Downloading VLASS maps') as onwards:
        for row_ind, row in samp.iterrows():
            # Read out the cluster name - so we can store things in the right place, and keep track of which image
            #  belongs to which cluster
            cur_name = row['name']
            
            # First, lets check if this cluster has had VLASS maps downloaded already. We'll use the project 
            #  history, as it is much computationally cheaper than checking if the expected file names exist
            if cur_history['data_operations'][cur_name]['raw_images']['vlass']['complete']:
                onwards.update(1)
                continue

            # Reading out relevant information - position is obviously very important, it is where we will center the
            #  downloaded images
            cur_ra = row['cent_im_ra']
            cur_dec = row['cent_im_dec']
            # The conversion between arcmin and arcminutes we calculated earlier, note that retrieving from a row
            #  of a dataframe like this seems to render it a float again, rather than an Astropy quantity
            cur_ang_prop_ratio = Quantity(row['ang_prop_ratio'], 'kpc/arcmin')

            # Though we already calculated the final pixel sizes we need for the LS images, the download method
            #  wants the pixel scale and an angular side length specified seperately, so we have to convert the
            #  proper side-length to degrees
            cur_deg_side_length = (side_length / cur_ang_prop_ratio).to('deg')
    
            # Populate the raw image storage directory path, with the cluster name and the current mission name
            cur_download_dir = raw_im_pth.format(n=cur_name, m='vlass')
            # Set up the final raw image file name, with some useful info in it
            cur_file_name = "{n}_sidelength{sl}_pixscale{ps}.fits".format(n=cur_name, sl=str(side_length).replace(' ', ''), 
                                                                          ps='2.5arcsecperpix')
            # Combine the raw image storage path with the file name
            cur_download_pth = os.path.join(cur_download_dir, cur_file_name)
            
            # Unfortunately this downloader can raise a general exception rather than a specific one, so I have
            #  to account for that
            try:
                vlass_down.download(ra=cur_ra, dec=cur_dec, ddir=cur_download_dir, size=cur_deg_side_length.value, 
                                    consider_QA_rejected=True, crop=True)
            # I am aware that excepting both is pointless because Exception is the super-class, but it is to remind me
            except (HTTPError, Exception) as err:
                print(err)
                # Record that we tried to get the image, but it wasn't available
                hist_change = True
                data_op_hist[cur_name]['raw_images']['vlass']['unavailable'] = True
                data_op_hist[cur_name]['raw_images']['vlass']['complete'] = True
                onwards.update(1)
                continue
                
            # Change the name of the downloaded file to something more descriptive
            # down_file_name = 'VLASS_{r}_{d}_'.format(r='{:.6f}'.format(cur_ra), d='{:.6f}'.format(cur_dec))
            # poss_vlass_files = [f for f in os.listdir(cur_download_dir) if down_file_name in f]
            if len(poss_vlass_files) != 1:
                raise ValueError("There are multiple possible VLASS images for {n}.".format(n=cur_name))
            else:
                down_file_name = poss_vlass_files[0]
                
            down_file_path = os.path.join(cur_download_dir, down_file_name)
            os.rename(down_file_path, cur_download_pth)
            
            # Now we add to the data operation history - as well as setting the change flag to True so we know to run the 
            #  history file update at the end of the looping
            hist_change = True
            data_op_hist[cur_name]['raw_images']['vlass']['complete'] = True
            # This shouldn't be necessary, as it should be setup so the sample content won't change, but we'll
            #  save the central RA and Dec here as well, so all the WCS-related information will be in one place
            data_op_hist[cur_name]['raw_images']['vlass']['cen_pos'] = [cur_ra, cur_dec]
            data_op_hist[cur_name]['raw_images']['vlass']['im_path'] = os.path.relpath(cur_download_pth)
            
            onwards.update(1)

    # # Only run the update history function if there were changes
    if hist_change:
        cur_history = update_history({'data_operations': data_op_hist})

os.chdir(TOP_WD)